In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
# strategy_version_input=705
# attribute_id_input=5131
formatted_attribute = 'thread_size'

In [3]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [4]:
import json
dfs = query_from_file(file_name='./query/custom_fields.sql', params=params)
custom_field_df=pd.json_normalize(dfs['custom_fields'])
field='ThreadSize'
fields = [field]
# df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df], axis = 1)
df = pd.concat([dfs.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
print(len(df))

del df['url']
del df['image_url']
del df['value']
del df['product_id']
del df['bucket_id']

4907


In [5]:
df[field].explode().value_counts()
# df.head(0)

3/4-16           222
9/16-18          202
7/8-14           176
1-1/16-12        122
1/2-14           119
13/16-16         108
1-3/16-12        106
5/8-18           106
7/16-20          105
1/2-20           104
11/16-16         104
1-5/16-12         92
1/4-18            83
1"-14             81
3/8-18            80
3/4-14            76
M18x1.5           54
M16x1.5           53
1-7/16-12         49
1-5/8-12          46
M30x2.0           41
M22x1.5           40
7/16-24           39
1-11/16-12        31
M14x1.5           29
3/8"              27
3/8-19            26
1-7/16"-12        26
7/8"-14           25
1/2"              25
1/8-27            25
M24x1.5           24
1"-11½            24
1/4"              24
M20x1.5           24
3/4-18            23
1-14              22
M26x1.5           21
3/4"              20
1-1/16-14         20
1-3/16"-12        19
1-1/16"-12        19
1/4-19            18
M36x2.0           18
G 1/2-14          17
1-11/16"-12       17
G 3/8-19          15
G 3/4-14     

In [6]:
import collections
print([item for item, count in collections.Counter(df['external_id']).items() if count > 1])

[]


In [31]:
# start=r'(?i)(^.{0,10})|()'
# df['start']=df['product_name'].apply(lambda x: re_extract(start, str(x)))

pattern=r'''(\d*.?\d+\/\d+\-\d+)|()''' 
df['name'] = df['product_name'].apply(lambda x: re_extract(pattern, str(x)))
df['ld'] = df['long_desc'].apply(lambda x: re_extract(pattern, str(x)))



pat=r'(\d*.?\d+\/\d+\-\d+)|()'
df['matches']=df[field].apply(lambda x: re_extract(pat, str(x)))

# pat=r'(?i)(m)|()'
# df['mm']=df[field].apply(lambda x: re_extract(pat, str(x)))


custom=df[(df[field].astype(str)!='nan')]
custom[curation_col]=custom['ThreadSize'].apply(lambda x: re.sub(r'.+to\s?|','',str(x))).apply(lambda x: re.sub(r"'|\[|\]",'',str(x))).apply(lambda x: re.sub(r'(?i)\s?mm',' mm',str(x))).apply(lambda x: re.sub(r'\s?\-\s?',' x ',str(x))).apply(lambda x: re.sub(r'"?\s?\-"?','-',str(x))).apply(lambda x: re.sub(r'"','',str(x)))#.apply(lambda x: re.sub(r'(?i)\s?mm\s?',' mm',str(x))).apply(lambda x: re.sub(r'\s?"\s?',' in',str(x))).apply(lambda x: f'{x} in').apply(lambda x: re.sub(r'\s?in\s?in',' in',str(x)))                                          
# custom[curation_col]=custom[field].apply(lambda x: f'{x}"').apply(lambda x: re.sub(r'""','"',str(x)))
print(len(custom))
matchcustom=custom[['external_id',curation_col]]
matchcustom[curation_col]=matchcustom[curation_col]
matchcustom=matchcustom[matchcustom[curation_col].astype(str)!='']
matchcustom[curation_col].explode().value_counts()

3025


3/4 x 16              222
9/16 x 18             202
7/8 x 14              201
1 x 1/16 x 12         141
1 x 3/16 x 12         125
1/2 x 14              119
13/16 x 16            108
5/8 x 18              106
7/16 x 20             105
1/2 x 20              104
1 x 5/16 x 12         104
11/16 x 16            104
1 x 14                103
1/4 x 18               83
3/8 x 18               80
3/4 x 14               76
1 x 7/16 x 12          75
M18x1.5                54
M16x1.5                53
1 x 5/8 x 12           52
1 x 11/16 x 12         48
M30x2.0                41
M22x1.5                40
7/16 x 24              39
M14x1.5                29
3/8                    27
1 x 1/16 x 14          27
3/8 x 19               26
1/2                    25
1/8 x 27               25
M24x1.5                24
M20x1.5                24
1 x 11½                24
1/4                    24
3/4 x 18               23
M26x1.5                21
3/4                    20
M36x2.0                18
1/4 x 19    

In [32]:
not_match=df[(df[field].astype(str)=='nan')]
# print(len(not_match[(not_match['ld'].astype(str)=='[]')|(not_match['name'].astype(str)!='[]')]))
two=not_match[(not_match['ld'].astype(str)!='[]')|(not_match['name'].astype(str)!='[]')]
# print(len(two))
# two[curation_col]=two['name'].apply(lambda x: re.sub(r'\s?\-\s?',' x ',str(x))).apply(lambda x: re.sub(r"'",'"',str(x)))
# matchtwo=two[['external_id',curation_col]]
# print(len(matchtwo))
two

buckets external_id  \
240  Compressors Accessories & Replacement Parts       56126   
259  Compressors Accessories & Replacement Parts       56129   
264  Compressors Accessories & Replacement Parts       56120   
270  Compressors Accessories & Replacement Parts       56123   

                                               product_name  \
240    1/2 E-Z Clip Splicer with 7/16-20 Thread Switch Port   
259    5/8 E-Z Clip Splicer with 7/16-20 Thread Switch Port   
264   5/16 E-Z Clip Splicer with 7/16-20 Thread Switch Port   
270  13/32 E-Z Clip Splicer with 7/16-20 Thread Switch Port   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                      long_desc  \
240  TYPICAL APPLICATION - Transportation, air conditioning & refrigeration systems. PRESSURE - Determined by hose burst pressure. ADVANTAGES - No crimp system provides the fastest, easiest, most reliable method for producing hose assemblies. ASSEMBLE WITH - E-Z Clip Pliers and Weatherhead GH134 hose. NO GUESSWORK NO CRIMPING OR LEAKY CRIMPS AS EASY TO USE AS A PAIR OF PLIERS INSTALLS EASILY - EVEN IN CONFINED AREAS CONNECTIONS EXCEED SAE J2064 SPECIFICATIONS   
259  TYPICAL APPLICATION - Transportation, air conditioning & refrigeration systems. PRESSURE - Determined by hose burst pressure. ADVANTAGES - No crimp system provides the fastest, easiest, most reliable method for producing hose assemblies. ASSEMBLE WITH - E-Z Clip Pliers and Weatherhead GH134 hose. NO GUESSWORK NO CRIMPING OR LEAKY CRIMPS AS EASY TO USE AS A PAIR OF PLIERS INSTALLS EASILY - EVEN IN CONFINED AREAS CONNECTIONS EXCEED SAE J2064 SPECIFICATIONS   
264  TYPICAL APPLICATION - Transportation, air conditioning & refrigeration systems. PRESSURE - Determined by hose burst pressure. ADVANTAGES - No crimp system provides the fastest, easiest, most reliable method for producing hose assemblies. ASSEMBLE WITH - E-Z Clip Pliers and Weatherhead GH134 hose. NO GUESSWORK NO CRIMPING OR LEAKY CRIMPS AS EASY TO USE AS A PAIR OF PLIERS INSTALLS EASILY - EVEN IN CONFINED AREAS CONNECTIONS EXCEED SAE J2064 SPECIFICATIONS   
270  TYPICAL APPLICATION - Transportation, air conditioning & refrigeration systems. PRESSURE - Determined by hose burst pressure. ADVANTAGES - No crimp system provides the fastest, easiest, most reliable method for producing hose assemblies. ASSEMBLE WITH - E-Z Clip Pliers and Weatherhead GH134 hose. NO GUESSWORK NO CRIMPING OR LEAKY CRIMPS AS EASY TO USE AS A PAIR OF PLIERS INSTALLS EASILY - EVEN IN CONFINED AREAS CONNECTIONS EXCEED SAE J2064 SPECIFICATIONS   

    ThreadSize        name  ld matches  
240        NaN  [ 7/16-20]  []      []  
259        NaN  [ 7/16-20]  []      []  
264        NaN  [ 7/16-20]  []      []  
270        NaN  [ 7/16-20]  []      []

In [33]:
# df[(df['name'].astype(str)!='[]')&(df['matches'].astype(str)!='[]')][0:500]

In [34]:
# two=df[(df['matches'].astype(str)=='[]')]
# print(len(two[two['name'].astype(str)!='[]']))
# two=two[two['name'].astype(str)!='[]']
# two[curation_col]=two['name'].apply(lambda x:re.sub(r'(?i)P\.?S\.?I\.?','',str(x))).apply(lambda x: re.sub(r'.+\-\s?',"['",str(x))).apply(lambda x: re.sub(r'(?<!\,)000',',000',str(x))).apply(lambda x: re.sub(r"\['",'["',str(x))).apply(lambda x: re.sub(r"\s?'\]",'"]',str(x))).apply(lambda x: re.sub(r"\s?\s?\s?'\s?\,\s?'",'","',str(x))).apply(lambda x: re.sub(r'200"\,"125','125","200',str(x))).apply(lambda x: re.sub(r'\["1\,500"\,"100"\,"2,000 "\]','["100","1,500","2,000"]',str(x))).apply(lambda x: re.sub(r'\s?"\]','"]',str(x)))                                                    
# two[curation_col].explode().value_counts()
# matchtwo=two[['external_id',curation_col]]

In [35]:
# two=df[(df['matches'].astype(str)=='[]')]
# # two['name']=two['name'].apply(lambda x: re.sub(r'(?i)\(|x','',str(x))).apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'\s?\,\s?',',',str(x))).apply(lambda x: re.sub(r'"\]','""]',str(x))).apply(lambda x:re.sub(r'\[""|\["','["',str(x)))                                                                  
# pat=r'(?i)((?i).{0,20}dia.{0,20})|()'
# two['name']=two['product_name'].apply(lambda x: re_extract(pat, str(x)))
# two['long']=two['long_desc'].apply(lambda x: re_extract(pat, str(x)))
# two[(two['name'].astype(str)!='[]')|(two['long'].astype(str)!='[]')]

# N/A

In [36]:
na=df[(df['name'].astype(str)=='[]')&(df[field].astype(str)=='nan')&(df['ld'].astype(str)=='[]')]
print(len(na))
na[curation_col]='n/a'
matchna=na[['external_id',curation_col]]
# print(len(na)+len(matchcustom)+len(matchtwo))
na

1878


buckets  \
0     Carrying & Lifting Equipment Replacement Parts & Accessories   
1                      Compressors Accessories & Replacement Parts   
2                      Compressors Accessories & Replacement Parts   
3                      Compressors Accessories & Replacement Parts   
4                      Compressors Accessories & Replacement Parts   
...                                                            ...   
4902                                      Hose Tools & Attachments   
4903                                      Hose Tools & Attachments   
4904                                      Hose Tools & Attachments   
4905                                      Hose Tools & Attachments   
4906                                      Hose Tools & Attachments   

     external_id                            product_name  \
0         419053                      3/8 Cold Shut Link   
1          40511            1/16 Viton O-Ring Assortment   
2          40490  1/16 - 1/8 Buna-N 90 O-Ring Assortment   
3          40492        3/32 Buna-N 90 O-Ring Assortment   
4          40491        1/16 Buna-N 90 O-Ring Assortment   
...          ...                                     ...   
4902     SPKHD12   3/4 Extra Heavy Duty Split Flange Kit   
4903     SPKHD16   3/4 Extra Heavy Duty Split Flange Kit   
4904     SPKHD20   3/4 Extra Heavy Duty Split Flange Kit   
4905     SPKHD24   3/4 Extra Heavy Duty Split Flange Kit   
4906     SPKHD32   3/4 Extra Heavy Duty Split Flange Kit   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          long_desc  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               Use with Grade 28 chain. Use next larger size than diameter of chain to insure extra factor of safety. Zinc plated.   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                     75 Durometer. Manufactured from fluorocarbon rubber. Temperature range: -20° to 400°F. Has maximum resistance to deterioration by heat and fluids. Recognized by brown color.   
2                                                                                                                                                                                                                                                                                                                                                                                                                         90 Durometer for high pressure applications. 90 Durometer recognized by yellow stripe. Manufactured from nitrile rubber. Temperature range: -65° to 300°F. Resists wide range of solvents, oils & fluids.   
3                                    

In [37]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcustom))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchna))
# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchtwo))

# def get_df_name(df):
#     name =[x for x in gglobals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchfinal))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchex))

matchcustom
matchna


In [15]:
stop

NameError: name 'stop' is not defined

# send to the folder for upload

In [38]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchcustom) 

In [39]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchna) 

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/Kimball MidWest - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('Kimball MidWest', attribute, df, matchtwo) 